In [1]:
import glob, os
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, mean_absolute_error

# from random import seed
# from random import randint

In [2]:
df = pd.read_csv("training_data.csv")

C:\Users\bune1\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (6,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
pd.set_option('display.max_columns', None)

In [4]:
train_df = df
train_df.head()

,PolicyNo,Policy_Company,Policy_Installment_Term,Policy_Billing_Code,Policy_Method_Of_Payment,Policy_Reinstatement_Fee_Indicator,Policy_Zip_Code_Garaging_Location,Vehicle_Territory,Vehicle_Make_Year,Vehicle_Make_Description,Vehicle_Performance,Vehicle_New_Cost_Amount,Vehicle_Symbol,Vehicle_Number_Of_Drivers_Assigned,Vehicle_Usage,Vehicle_Miles_To_Work,Vehicle_Days_Per_Week_Driven,Vehicle_Annual_Miles,Vehicle_Anti_Theft_Device,Vehicle_Passive_Restraint,Vehicle_Age_In_Years,Vehicle_Med_Pay_Limit,Vehicle_Bodily_Injury_Limit,Vehicle_Physical_Damage_Limit,Vehicle_Comprehensive_Coverage_Indicator,Vehicle_Comprehensive_Coverage_Limit,Vehicle_Collision_Coverage_Indicator,Vehicle_Collision_Coverage_Deductible,Driver_Total,Driver_Total_Male,Driver_Total_Female,Driver_Total_Single,Driver_Total_Married,Driver_Total_Related_To_Insured_Self,Driver_Total_Related_To_Insured_Spouse,Driver_Total_Related_To_Insured_Child,Driver_Total_Licensed_In_State,Driver_Minimum_Age,Driver_Maximum_Age,Driver_Total_Teenager_Age_15_19,Driver_Total_College_Ages_20_23,Driver_Total_Young_Adult_Ages_24_29,Driver_Total_Low_Middle_Adult_Ages_30_39,Driver_Total_Middle_Adult_Ages_40_49,Driver_Total_Adult_Ages_50_64,Driver_Total_Senior_Ages_65_69,Driver_Total_Upper_Senior_Ages_70_plus,Vehicle_Youthful_Driver_Indicator,Vehicle_Youthful_Driver_Training_Code,Vehicle_Youthful_Good_Student_Code,Vehicle_Driver_Points,Vehicle_Safe_Driver_Discount_Indicator,EEA_Liability_Coverage_Only_Indicator,EEA_Multi_Auto_Policies_Indicator,EEA_Policy_Zip_Code_3,EEA_Policy_Tenure,EEA_Agency_Type,EEA_Packaged_Policy_Indicator,EEA_Full_Coverage_Indicator,EEA_Prior_Bodily_Injury_Limit,EEA_PolicyYear,SYS_Renewed,SYS_New_Business,Annual_Premium,Claim_Count,Loss_Amount,Frequency,Severity,Loss_Ratio
0,164532941,Standard,6,Direct Billed to Insured,Pre-paid,N,43046,35,2004,BUIK LESABRE LI,Standard,-1,11,1,Farm,-1,5,Unknown,Not Applicable,Y,3,1000,250-500,100000,N,-1,Y,500,1,0,1,0,0,1,0,0,1,75,75,0,0,0,0,0,0,0,1,N,Not Applicable,Not Eligible for Good Student Credit,0,Y,N,N,430,22.7,Standard,N,Y,100-400,2006,Y,N,320.12,0,0.0,0.0,0.0,0.0
1,164533241,Standard,6,Direct Billed to Insured,Pre-paid,N,Unknown,35,1980,CADILLAC 4-DOOR,Standard,-1,11,1,Farm,-1,5,Unknown,Not Applicable,N,9,2000,100-300,50000,N,-1,Y,50,1,0,1,0,1,0,0,0,1,71,71,0,0,0,0,0,0,0,1,N,Not Applicable,Not Eligible for Good Student Credit,0,Y,N,N,Unknown,47.1,Preferred,N,Y,100-200,2006,Y,N,259.70,0,0.0,0.0,0.0,0.0
2,164534633,Standard,6,Direct Billed to Insured,Pre-paid,N,43555,17,2005,PONT MONTANA SV,Standard,-1,10,1,Pleasure,-1,5,Unknown,Passive Disabling-Vehicle Recovery,Y,2,50000,250-500,100000,N,-1,Y,100,1,1,0,0,1,1,0,0,1,80,80,0,0,0,0,0,0,0,1,N,Not Applicable,Not Eligible for Good Student Credit,0,Y,N,N,435,47.2,Non-standard,N,Y,100-400,2006,Y,N,613.74,0,0.0,0.0,0.0,0.0
3,164534839,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,Standard,-1,12,1,Pleasure,-1,5,Unknown,Passive Disabling-Vehicle Recovery,Y,2,5000,50-100,35000,N,-1,Y,250,1,1,0,0,1,1,0,0,1,74,74,0,0,0,0,0,0,0,1,N,Not Applicable,Not Eligible for Good Student Credit,0,Y,N,Y,435,46.7,Non-standard,Y,Y,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0
4,164534840,Standard,6,Direct Billed to Insured,Pre-paid,N,43561,17,2005,MERC GRAND MARQ,Standard,-1,12,1,Pleasure,-1,5,Unknown,Passive Disabling-Vehicle Recovery,Y,2,5000,50-100,35000,N,-1,Y,250,1,1,0,0,1,1,0,0,1,75,75,0,0,0,0,0,0,0,1,N,Not Applicable,Not Eligible for Good Student Credit,0,Y,N,Y,435,47.2,Non-standard,Y,Y,40-100,2006,Y,N,541.66,0,0.0,0.0,0.0,0.0


In [5]:
# cols = df.columns.tolist()
# cols

In [43]:
start_drop = ['Policy_Reinstatement_Fee_Indicator','Policy_Zip_Code_Garaging_Location','Vehicle_Make_Year','Vehicle_Make_Description',
            'Vehicle_New_Cost_Amount','Vehicle_Symbol','Vehicle_Number_Of_Drivers_Assigned','Vehicle_Days_Per_Week_Driven',
            'Vehicle_Annual_Miles','Vehicle_Anti_Theft_Device','Vehicle_Passive_Restraint','Vehicle_Med_Pay_Limit',
            'Vehicle_Bodily_Injury_Limit','Vehicle_Physical_Damage_Limit','Vehicle_Comprehensive_Coverage_Indicator',
            'Vehicle_Comprehensive_Coverage_Limit','Vehicle_Collision_Coverage_Indicator','Vehicle_Collision_Coverage_Deductible',
            'Driver_Total_Related_To_Insured_Self','Driver_Total_Related_To_Insured_Spouse','Driver_Total_Related_To_Insured_Child',
            'Driver_Total_Licensed_In_State','Driver_Minimum_Age','Driver_Maximum_Age','Vehicle_Youthful_Driver_Indicator',
            'Vehicle_Youthful_Driver_Training_Code','Vehicle_Safe_Driver_Discount_Indicator','EEA_Liability_Coverage_Only_Indicator',
            'EEA_Multi_Auto_Policies_Indicator','EEA_Policy_Zip_Code_3','EEA_Policy_Tenure','EEA_Agency_Type','EEA_Packaged_Policy_Indicator',
            'EEA_Full_Coverage_Indicator','EEA_Prior_Bodily_Injury_Limit','EEA_PolicyYear','SYS_Renewed','SYS_New_Business',
            'Claim_Count','Frequency','Severity','Loss_Ratio']

In [7]:
train_df = train_df.drop(start_drop,axis=1)

In [8]:
train_df.shape

(424431, 27)

In [9]:
df1 = train_df.loc[train_df['Loss_Amount'] == 0.0]
df2 = train_df.loc[train_df['Loss_Amount'] != 0.0]

In [10]:
df1.shape

(405562, 27)

In [11]:
df2.shape

(18869, 27)

## Portfolio Creation

In [12]:
# save_traingset_count = 1
perlist = [1,2,3,4,5,6,7,8,9,10,20]
divlist = [1000,3000,5000]
index_divlist = len(divlist) #3
index_perlist = len(perlist) #11

In [66]:
def createPortfolio(a,b,count):
    i=divlist[a]
    j=perlist[b]
    lossamount = round((i*j)/100)
    nolossamount = i - lossamount
    frames = [df1.sample(n=nolossamount), df2.sample(n=lossamount)]
    result = pd.concat(frames)
    result = shuffle(result)
    return result

In [14]:
def percentagemaker(df1,df2):
    columnName = [] #create list for different column names
    getcontent = [] #another list to append the percentages of the total
    for col in df1.columns:
        total = df1[col].count()
        if(col!='Index' and col!='Annual_Premium' and col!='Loss_Amount' and col!='Vehicle_Miles_To_Work'):
            checkuni = df1[col].unique()
            for c in checkuni:
                columnName.append(col + "__" + str(c) + "__%")
                totaluniquecount = df1[df1[col] == c].count()['Index']
                calculatePer = (totaluniquecount/total)*100
                getcontent.append(calculatePer)
        if(col=='Vehicle_Miles_To_Work'):
            columnName.append(col)
            getcontent.append(df1[col].mean())
        if(col=='Annual_Premium'):
            columnName.append(col)
            getcontent.append(df1[col].sum())
        if(col=='Loss_Amount'):
            columnName.append(col)
            getcontent.append(df1[col].sum())
        
        
    tempdf = pd.DataFrame(columns = columnName)
    df2 = pd.concat([df2,tempdf])
    df2 = df2.append(pd.Series(getcontent, index=columnName), ignore_index=True)
    return df2

In [15]:
df_toTrain = pd.DataFrame()

divlist_counter = 0
perlist_counter = 0

for k in range(1,331):
    if((k-1)%10 == 0 and (k-1) != 0):
        if(divlist_counter==index_divlist-1):
            divlist_counter=0
        else:
            divlist_counter=divlist_counter+1

        if(perlist_counter==index_perlist-1):
            perlist_counter=0
        else:
            perlist_counter=perlist_counter+1
    
    df1000 = createPortfolio(divlist_counter,perlist_counter,k)
    df1000 = df1000.reset_index(drop=True)
    df1000 = df1000.drop(['PolicyNo'], axis=1)
    df1000['Index'] = df1000.index
    df_toTrain = percentagemaker(df1000,df_toTrain)
    
# filename = "training_portfolio_df1000_.csv"
# df_toTrain.to_csv(filename)

C:\Users\bune1\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [45]:
df_toTrain.head()

,Annual_Premium,Driver_Total_Adult_Ages_50_64__0__%,Driver_Total_Adult_Ages_50_64__1__%,Driver_Total_Adult_Ages_50_64__2__%,Driver_Total_College_Ages_20_23__0__%,Driver_Total_College_Ages_20_23__1__%,Driver_Total_College_Ages_20_23__2__%,Driver_Total_Female__0__%,Driver_Total_Female__1__%,Driver_Total_Female__2__%,Driver_Total_Low_Middle_Adult_Ages_30_39__0__%,Driver_Total_Low_Middle_Adult_Ages_30_39__1__%,Driver_Total_Low_Middle_Adult_Ages_30_39__2__%,Driver_Total_Male__0__%,Driver_Total_Male__1__%,Driver_Total_Male__2__%,Driver_Total_Married__0__%,Driver_Total_Married__1__%,Driver_Total_Married__2__%,Driver_Total_Middle_Adult_Ages_40_49__0__%,Driver_Total_Middle_Adult_Ages_40_49__1__%,Driver_Total_Middle_Adult_Ages_40_49__2__%,Driver_Total_Senior_Ages_65_69__0__%,Driver_Total_Senior_Ages_65_69__1__%,Driver_Total_Senior_Ages_65_69__2__%,Driver_Total_Single__0__%,Driver_Total_Single__1__%,Driver_Total_Single__2__%,Driver_Total_Teenager_Age_15_19__0__%,Driver_Total_Teenager_Age_15_19__1__%,Driver_Total_Teenager_Age_15_19__2__%,Driver_Total_Upper_Senior_Ages_70_plus__0__%,Driver_Total_Upper_Senior_Ages_70_plus__1__%,Driver_Total_Upper_Senior_Ages_70_plus__2__%,Driver_Total_Young_Adult_Ages_24_29__0__%,Driver_Total_Young_Adult_Ages_24_29__1__%,Driver_Total_Young_Adult_Ages_24_29__2__%,Driver_Total__1__%,Driver_Total__2__%,Loss_Amount,Policy_Billing_Code__Direct Billed to Insured__%,Policy_Billing_Code__Premium Finance__%,Policy_Company__Preferred__%,Policy_Company__Standard __%,Policy_Installment_Term__12__%,Policy_Installment_Term__6__%,Policy_Method_Of_Payment__Installment__%,Policy_Method_Of_Payment__Pre-paid__%,Vehicle_Age_In_Years__1__%,Vehicle_Age_In_Years__2__%,Vehicle_Age_In_Years__3__%,Vehicle_Age_In_Years__4__%,Vehicle_Age_In_Years__5__%,Vehicle_Age_In_Years__6__%,Vehicle_Age_In_Years__7__%,Vehicle_Age_In_Years__8__%,Vehicle_Age_In_Years__9__%,Vehicle_Driver_Points__0__%,Vehicle_Driver_Points__1__%,Vehicle_Driver_Points__2__%,Vehicle_Driver_Points__3__%,Vehicle_Driver_Points__4__%,Vehicle_Miles_To_Work,Vehicle_Performance__High__%,Vehicle_Performance__Intermediate__%,Vehicle_Performance__Sports Premium__%,Vehicle_Performance__Sports__%,Vehicle_Performance__Standard__%,Vehicle_Territory__13__%,Vehicle_Territory__15__%,Vehicle_Territory__16__%,Vehicle_Territory__17__%,Vehicle_Territory__18__%,Vehicle_Territory__19__%,Vehicle_Territory__20__%,Vehicle_Territory__22__%,Vehicle_Territory__26__%,Vehicle_Territory__27__%,Vehicle_Territory__30__%,Vehicle_Territory__31__%,Vehicle_Territory__32__%,Vehicle_Territory__35__%,Vehicle_Territory__36__%,Vehicle_Territory__37__%,Vehicle_Usage__Business__%,Vehicle_Usage__Farm __%,Vehicle_Usage__Pleasure__%,Vehicle_Usage__Work __%,Vehicle_Youthful_Good_Student_Code__Eligible for Good Student Credit __%,Vehicle_Youthful_Good_Student_Code__Not Eligible for Good Student Credit__%
0,326529.38,70.9,26.7,2.4,95.0,5.0,0.0,43.7,55.5,0.8,79.9,18.9,1.2,45.4,53.9,0.7,39.4,50.9,9.7,75.1,24.0,0.9,94.7,4.8,0.5,76.4,23.0,0.6,96.3,3.6,0.1,90.6,8.9,0.5,91.2,8.3,0.5,87.6,12.4,54079.83,97.7,2.3,7.7,92.3,3.5,96.5,51.2,48.8,2.7,5.6,5.1,4.9,7.6,7.2,6.6,6.4,53.9,91.5,7.7,0.7,0.1,0.0,3.167,1.0,2.7,0.5,0.2,95.6,1.8,0.4,0.8,0.7,0.5,0.5,0.7,0.4,0.7,4.2,17.3,42.2,1.9,24.4,0.8,2.7,0.9,7.4,57.2,34.5,1.2,98.8
1,334637.33,71.9,26.4,1.7,96.7,3.2,0.1,42.0,57.5,0.5,80.4,18.4,1.2,48.3,50.4,1.3,39.0,52.2,8.8,74.8,23.2,2.0,94.0,5.6,0.4,77.3,22.5,0.2,95.4,4.6,0.0,90.3,9.1,0.6,91.4,8.2,0.4,88.5,11.5,40316.79,98.4,1.6,6.4,93.6,3.8,96.2,48.7,51.3,4.4,3.9,5.4,6.7,6.0,7.0,6.2,6.3,54.1,91.4,6.9,1.5,0.2,0.0,3.589,1.2,2.8,0.3,0.1,95.6,1.1,0.4,0.7,1.8,0.5,0.1,0.4,0.1,0.6,4.0,17.9,42.0,1.9,24.5,0.7,3.3,0.2,9.1,50.9,39.8,1.5,98.5
2,327950.89,72.8,25.2,2.0,94.8,5.0,0.2,41.7,57.7,0.6,79.0,19.7,1.3,47.7,51.8,0.5,38.2,53.1,8.7,73.9,25.3,0.8,93.6,6.2,0.2,78.0,21.2,0.8,96.5,3.5,0.0,91.1,8.0,0.9,92.6,6.8,0.6,88.3,11.7,63179.63,98.2,1.8,5.8,94.2,3.5,96.5,50.4,49.6,3.7,4.6,5.8,5.7,6.1,6.2,7.8,7.7,52.4,90.4,8.2,1.3,0.1,0.0,3.068,1.4,2.9,0.7,0.3,94.7

In [46]:
filterValues = []
cols = df_toTrain.columns.tolist()
for column in df_toTrain.columns:    
    i = df_toTrain[column].isnull().sum()
#     print(i)
    if(i > 170):
        filterValues.append(column)
# print(filterValues)

In [47]:
df_toTrain = df_toTrain.drop(filterValues, axis=1)

In [48]:
df_toTrain = df_toTrain.fillna(0)

In [49]:
X = df_toTrain.drop('Loss_Amount',axis=1)
y = df_toTrain['Loss_Amount']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
regressor = RandomForestRegressor(n_estimators=100)
rf = regressor.fit(X_train,y_train)
regressor.score(X_train,y_train)

0.974257490969239

In [44]:
path = 'testing_portfolios'
all_files = glob.glob(os.path.join(path, "*.csv"))  

file_names = []
log_LR_pred_list = []
log_LR_list = []

df_toTest = pd.DataFrame()

start_drop.remove('Claim_Count')
start_drop.remove('Frequency')
start_drop.remove('Severity')
start_drop.remove('Loss_Ratio')

filename_list = []
filename_count = []

for file in all_files:
    new_filename = str(file)
    df_from_each_file = pd.read_csv(file)
    file = os.path.split(file)
    file = file[1]
    file_names.append(file.split('.')[0][5:])
    
    df_from_each_file = df_from_each_file.drop(start_drop, axis=1)

    filename = str(new_filename)
    filename_split = filename.split('\\')
    filename_split_2 = filename_split[1].split('.')
    filename_split_3 = filename_split_2[0].split('_')
    filename_list.append(filename_split_3[1]+"_"+filename_split_3[2])
    filename_count.append(filename_split_3[2])    
    
    df_from_each_file = df_from_each_file.reset_index(drop=True)
    df_from_each_file = df_from_each_file.drop(['PolicyNo'], axis=1)
    df_from_each_file['Index'] = df_from_each_file.index
    df_toTest = percentagemaker(df_from_each_file,df_toTest)

# filename = "testing_portfolio_main_" + str(randint(0, 1000)) + ".csv"
# df_toTest.to_csv(filename)
    

C:\Users\bune1\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [51]:
df_toTest.head()

,Annual_Premium,Driver_Total_Adult_Ages_50_64__0__%,Driver_Total_Adult_Ages_50_64__1__%,Driver_Total_Adult_Ages_50_64__2__%,Driver_Total_Adult_Ages_50_64__3__%,Driver_Total_Adult_Ages_50_64__4__%,Driver_Total_College_Ages_20_23__0__%,Driver_Total_College_Ages_20_23__1__%,Driver_Total_College_Ages_20_23__2__%,Driver_Total_Female__0__%,Driver_Total_Female__1__%,Driver_Total_Female__2__%,Driver_Total_Female__3__%,Driver_Total_Low_Middle_Adult_Ages_30_39__0__%,Driver_Total_Low_Middle_Adult_Ages_30_39__1__%,Driver_Total_Low_Middle_Adult_Ages_30_39__2__%,Driver_Total_Low_Middle_Adult_Ages_30_39__3__%,Driver_Total_Male__0__%,Driver_Total_Male__1__%,Driver_Total_Male__2__%,Driver_Total_Male__3__%,Driver_Total_Married__0__%,Driver_Total_Married__1__%,Driver_Total_Married__2__%,Driver_Total_Married__3__%,Driver_Total_Married__4__%,Driver_Total_Middle_Adult_Ages_40_49__0__%,Driver_Total_Middle_Adult_Ages_40_49__1__%,Driver_Total_Middle_Adult_Ages_40_49__2__%,Driver_Total_Middle_Adult_Ages_40_49__3__%,Driver_Total_Senior_Ages_65_69__0__%,Driver_Total_Senior_Ages_65_69__1__%,Driver_Total_Senior_Ages_65_69__2__%,Driver_Total_Single__0__%,Driver_Total_Single__1__%,Driver_Total_Single__2__%,Driver_Total_Single__3__%,Driver_Total_Teenager_Age_15_19__0__%,Driver_Total_Teenager_Age_15_19__1__%,Driver_Total_Teenager_Age_15_19__2__%,Driver_Total_Upper_Senior_Ages_70_plus__0__%,Driver_Total_Upper_Senior_Ages_70_plus__1__%,Driver_Total_Upper_Senior_Ages_70_plus__2__%,Driver_Total_Young_Adult_Ages_24_29__0__%,Driver_Total_Young_Adult_Ages_24_29__1__%,Driver_Total_Young_Adult_Ages_24_29__2__%,Driver_Total_Young_Adult_Ages_24_29__3__%,Driver_Total__1__%,Driver_Total__2__%,Driver_Total__3__%,Driver_Total__4__%,Policy_Billing_Code__Direct Billed to Insured__%,Policy_Billing_Code__Premium Finance__%,Policy_Company__Preferred__%,Policy_Company__Standard __%,Policy_Installment_Term__12__%,Policy_Installment_Term__6__%,Policy_Method_Of_Payment__Installment__%,Policy_Method_Of_Payment__Pre-paid__%,Vehicle_Age_In_Years__1__%,Vehicle_Age_In_Years__2__%,Vehicle_Age_In_Years__3__%,Vehicle_Age_In_Years__4__%,Vehicle_Age_In_Years__5__%,Vehicle_Age_In_Years__6__%,Vehicle_Age_In_Years__7__%,Vehicle_Age_In_Years__8__%,Vehicle_Age_In_Years__9__%,Vehicle_Driver_Points__0__%,Vehicle_Driver_Points__1__%,Vehicle_Driver_Points__2__%,Vehicle_Driver_Points__3__%,Vehicle_Driver_Points__4__%,Vehicle_Driver_Points__5__%,Vehicle_Driver_Points__6__%,Vehicle_Driver_Points__7__%,Vehicle_Miles_To_Work,Vehicle_Performance__High__%,Vehicle_Performance__Intermediate__%,Vehicle_Performance__Sports Premium__%,Vehicle_Performance__Sports__%,Vehicle_Performance__Standard__%,Vehicle_Territory__13__%,Vehicle_Territory__15__%,Vehicle_Territory__16__%,Vehicle_Territory__17__%,Vehicle_Territory__18__%,Vehicle_Territory__19__%,Vehicle_Territory__20__%,Vehicle_Territory__22__%,Vehicle_Territory__26__%,Vehicle_Territory__27__%,Vehicle_Territory__30__%,Vehicle_Territory__31__%,Vehicle_Territory__32__%,Vehicle_Territory__35__%,Vehicle_Territory__36__%,Vehicle_Territory__37__%,Vehicle_Usage__Business__%,Vehicle_Usage__Farm __%,Vehicle_Usage__Pleasure__%,Vehicle_Usage__Work __%,Vehicle_Youthful_Good_Student_Code__Eligible for Good Student Credit __%,Vehicle_Youthful_Good_Student_Code__Not Eligible for Good Student Credit__%
0,331804.80,69.300000,28.800000,1.900000,NaN,NaN,95.600000,4.400000,NaN,41.900000,56.700000,1.400000,NaN,79.800000,18.600000,1.600000,NaN,47.300000,52.100000,0.600000,NaN,38.500000,52.800000,8.700000,NaN,NaN,74.300000,24.000000,1.700000,NaN,94.100000,5.700000,0.200000,76.000000,23.400000,0.600000,NaN,96.000000,4.000000,NaN,91.900000,7.700000,0.400000,92.300000,7.400000,0.300000,NaN,87.200000,12.800000,NaN,NaN,98.000000,2.000000,6.800000,93.200000,3.800000,96.200000,52.20000,47.80000,3.200000,6.100000,5.700000,4.900000,6.000000,7.700000,6.200000,5.900000,54.300000,90.500000,8.000000,1.500000,NaN,NaN,NaN,NaN,NaN,2.996000,0.900000,2.800000,0.400000,0.3000,95.600000,1.500000,0.500000,1.

In [52]:
df_toTest = df_toTest.drop(filterValues, axis=1)

In [53]:
df_toTest = df_toTest.fillna(0)

In [54]:
set_Test = set(df_toTest.columns)
set_Train = set(X.columns)
s = set_Test.difference(set_Train)
main_vals = list(s)
df_toTest = df_toTest.drop(main_vals, axis=1)

In [55]:
predictions = rf.predict(df_toTest)

In [56]:
# print(predictions)

In [57]:
data = []
for i in range(0, len(predictions)):
    lr = math.log(predictions[i])-math.log(df_toTest.iloc[i]['Annual_Premium'])
    data.append([int(filename_count[i]), filename_list[i], lr])
df_preds = pd.DataFrame(data, columns= ['count', 'ID', 'ln_LR'])

In [58]:
df_preds = df_preds.sort_values('count')

In [59]:
df_preds.head()

,count,ID,ln_LR
0,1,portfolio_1,0.037292
60,2,portfolio_2,-0.332020
132,3,portfolio_3,0.473440
183,4,portfolio_4,0.015904
254,5,portfolio_5,0.201736


In [60]:
df_preds = df_preds.drop(['count'], axis=1)

In [61]:
df_preds.to_csv('predictions.csv', index=False)